<a href="https://colab.research.google.com/github/nanopiero/fusion/blob/main/notebooks/fcns/training_B11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## B2 radar + cmls -> pluvios 1 min + cmls [xrlg1_yg1l]


In [ ]:
! git clone https://github.com/nanopiero/fusion.git

In [8]:
! pwd

/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation/fusion/notebooks/fcns


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import os
import time
import sys
sys.path.append('/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation')

from fusion.utils.datasets import spatialized_gt, create_cmls_filter, FusionDataset
from fusion.utils.datasets import indices_to_sampled_values, get_point_measurements, point_gt, segment_gt, make_noisy_images
from torch.utils.data import DataLoader
from fusion.utils.fcn import UNet
from fusion.utils.cost_functions import QPELoss_fcn, compute_metrics
from fusion.utils.viz import set_tensor_values2, plot_images, plot_images_10pts_20seg, plot_results_10pts_20seg

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# Config
num_epochs = 1500
save_every = 1
path = r'/scratch/mdso/lepetitp/ppc/RAINCELL/models/simulation/checkpoint_fcn_exp_B21_xrl_yg1l.pt'
npairs = 40
nsteps = 60
ndiscs = 5
size_image=64
length_dataset = 6400
device = torch.device('cuda:0')

# Entraînement
npoints = 10
dataset = FusionDataset(length_dataset=length_dataset,
                        npairs=npairs,
                        nsteps=nsteps,
                        ndiscs=ndiscs, size_image=size_image)


loader = DataLoader(dataset, batch_size=64, num_workers=4)


In [5]:
# Tiny UNet V1. 60 new channels for input time series of rain gauges measurements
use_fcn = True
ch_in = 72 
ch_out = nsteps * 3 + 1
size = nsteps * 3

model = UNet(ch_in, ch_out, size, nb_additional_parameters=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


In [6]:
# Boucle avec 5 modes d'évaluation
val_steps = ['eval_opportunity_cost_cml_spat',
             'eval_added_value_half_cml_spat',
             'eval_added_value_full_cml_spat',
             ]
steps = val_steps + ['train']
losses = {step:[] for step in steps}
last_epoch = 0

In [7]:
def segment_gt(images, pairs, filters, use_fcn=False, split=None, return_sampled_values_and_filters=False):
  """
  split : list of tuples (min link id, max link id + 1)) 
  """
    
  bs, nsteps, S, _ = images.shape
  _, nlinks, _, _ = filters.shape

  if split is None :
      filtered_images = images.unsqueeze(dim=2) * \
                    filters.unsqueeze(dim=1)     
      sampled_values = torch.sum(filtered_images,\
                        dim=(3,4))
      denom = torch.sum(filters.unsqueeze(dim=1),\
                        dim=(3,4))
      sampled_values /= denom 
      segment_measurements = torch.cat((pairs, sampled_values), dim=1)
        
    
      if not use_fcn:
        return segment_measurements, None
      else:
        sampled_values += 0.1
        filters = filters.unsqueeze(1) > 0
        filters = filters * sampled_values.view(bs, nsteps, nlinks, 1, 1)
        filters = filters.sum(dim=2)
        
      return segment_measurements, filters

  else :
      list_segments = []
      
      for (l0, l1) in split:
          partial_filters = filters[:,l0:l1,...]
          
          filtered_images = images.unsqueeze(dim=2) * \
                            partial_filters.unsqueeze(dim=1)     
          sampled_values = torch.sum(filtered_images,\
                            dim=(3,4))
          denom = torch.sum(partial_filters.unsqueeze(dim=1),\
                            dim=(3,4))
          sampled_values /= denom 
          segment_measurements = torch.cat((pairs[:,:,l0:l1], sampled_values), dim=1)
            
        
          if not use_fcn:
              if return_sampled_values_and_filters:
                list_segments.append((segment_measurements, None, sampled_values, partial_filters))
              else:
                list_segments.append((segment_measurements, None))
          else:
            sampled_values += 0.1
            link_inputs = partial_filters.unsqueeze(1) > 0 
            link_inputs = link_inputs * sampled_values.view(bs, nsteps, l1 - l0, 1, 1)
            link_inputs = link_inputs.sum(dim=2)

            if return_sampled_values_and_filters:
                list_segments.append((segment_measurements, link_inputs, sampled_values, partial_filters))
            else:
                list_segments.append((segment_measurements, link_inputs))
                
      return list_segments
      

In [8]:
import torch.nn as nn
from sklearn.metrics import confusion_matrix


class QPELoss_fcn(nn.Module):
    def __init__(self, eval_qpe_1h = False, eval_independent_qpe_1h=False, eval_segments=False):
        super(QPELoss_fcn, self).__init__()
        self.regression_loss = nn.MSELoss()
        self.segmentation_loss = nn.CrossEntropyLoss()
        self.eval_qpe_1h = eval_qpe_1h
        self.eval_independent_qpe_1h = eval_independent_qpe_1h
        self.eval_segments = eval_segments

    def forward(self, p, outputs, targets, sampled_values_and_filters=None):

      # sans supervision imparfaite, avec spatialisation
      bs, nsteps, S, _ = targets.shape
      targets = targets.view(bs, 1, nsteps, S**2)
      mask = (targets>=0)
      masked_targets = targets[mask]
      outputs_rnr0 = outputs[:, :nsteps, ...].view(bs, 1, nsteps, S**2)
      outputs_rnr1 = outputs[:, nsteps:2*nsteps, ...].view(bs, 1, nsteps, S**2)
      masked_outputs_rnr = torch.cat([outputs_rnr0[mask].view(bs, 1, -1), outputs_rnr1[mask].view(bs, 1, -1)], dim=1)
      masked_targets_rnr =  (masked_targets > 0).long().view(bs, -1)
      outputs_qpe = outputs[:, 2*nsteps:3*nsteps , ...]\
                                .view(bs, 1, nsteps, S**2)

      masked_outputs_qpe = outputs_qpe[mask][masked_targets > 0]
      masked_targets_qpe = masked_targets[masked_targets > 0]

      loss_rnr = self.segmentation_loss(masked_outputs_rnr, masked_targets_rnr)
      loss_qpe_1min = self.regression_loss(masked_outputs_qpe, masked_targets_qpe)
      loss = 1/(2*p[0]**2) * loss_qpe_1min + 1/(2*p[1]**2) * loss_rnr
      

      if self.eval_qpe_1h:
        loss_qpe_1h = self.regression_loss(outputs_qpe.sum(dim=2)[mask[:, :, 0, ...]],
                                           targets.sum(dim=2)[mask[:, :, 0, ...]])
        loss += 1/(2*p[2]**2) * loss_qpe_1h

      elif self.eval_independent_qpe_1h:
        outputs_qpe_1h = outputs[:, 3*nsteps, ...]\
                                .view(bs, 1, S**2)
          
        loss_qpe_1h = self.regression_loss(outputs_qpe_1h[mask[:, :, 0, ...]],
                                           targets.sum(dim=2)[mask[:, :, 0, ...]])   
        loss += 1/(2*p[2]**2) * loss_qpe_1h
      else:
        with torch.no_grad():
            loss_qpe_1h = self.regression_loss(outputs_qpe.sum(dim=2)[mask[:, :, 0, ...]],
                                               targets.sum(dim=2)[mask[:, :, 0, ...]])
          
          
      if self.eval_segments:
          sampled_values, filters = sampled_values_and_filters
          #Pas très joli : on aurait aimé appler outputs_QPE, mais il a été applati
          filtered_outputs = outputs[:, 2*nsteps:3*nsteps, ...].unsqueeze(dim=2) * \
                             filters.unsqueeze(dim=1)
          sampled_pred_values = torch.sum(filtered_outputs,\
                            dim=(3,4))
          loss_segments = self.regression_loss(sampled_pred_values, sampled_values)
          loss += 1/(2*p[3]**2) * loss_segments
      else:
          loss_segments = 0.
      
      loss += torch.log(1 + p[0]**2 + p[1]**2 + self.eval_qpe_1h * p[2]**2 + self.eval_independent_qpe_1h * p[2]**2 + self.eval_segments * p[3]**2)

        
      with torch.no_grad():
        preds = masked_outputs_rnr.argmax(dim=1).flatten().cpu().numpy()
        targets = masked_targets_rnr.flatten().cpu().numpy()
        # Compute the confusion matrix
        cm = confusion_matrix(targets, preds, labels=np.arange(2))

      if self.eval_segments:
          return loss_qpe_1min.item(), loss_qpe_1h.item(), loss_rnr.item(), loss, cm, loss_segments.item()
      else:
          return loss_qpe_1min.item(), loss_qpe_1h.item(), loss_rnr.item(), loss, cm, loss_segments

criterion = QPELoss_fcn(eval_segments=True)

In [9]:
checkpoint = torch.load(path, \
                            map_location=device)
last_epoch = checkpoint['epoch']
losses = checkpoint['train_losses']
# best_loss = checkpoint['best_loss']
model_weights = checkpoint['model']
optimizer_state_dict = checkpoint['optimizer']
# scheduler_state_dict = checkpoint['scheduler']
model.load_state_dict(model_weights)
optimizer.load_state_dict(optimizer_state_dict)
# scheduler.load_state_dict(scheduler_state_dict)
del checkpoint, model_weights, optimizer_state_dict

In [10]:
last_epoch

973

In [ ]:
model.train()

for epoch in range(last_epoch, num_epochs + 1):
  t = time.time()
  print('epoch n°', epoch, '\n')

  running_regression_loss = {step:0.0 for step in steps}
  running_regression_loss_1h = {step:0.0 for step in steps}
  running_segmentation_loss = {step:0.0 for step in steps}
  running_confusion_matrix = {step: np.zeros((2, 2), dtype=int) for step in steps}
  running_integ_loss = {step:0.0 for step in steps}
    
  for i, (images, pairs, filters) in enumerate(loader):

    # ground truth (not usable)
    images = images.clone().detach().float().to(device)

    # pseudo radar
    noisy_images = make_noisy_images(images)

    # pseudo CMLs
    pairs = pairs.clone().detach().float().to(device)
    filters = filters.clone().float().detach().to(device)
      
    _, point_measurements_fcn, _ = point_gt(images, npoints=npoints, use_fcn=use_fcn)
    
    #Validation steps
    model.eval()
    with torch.no_grad():

      # 4 first val steps 
      # segment_measurements = segment_gt(images, pairs, filters)
      splitl_val = [(0,5),(0,10),(0,20),(20,40)]
      segment_measurements_fcn_val_few, segment_measurements_fcn_val_half, segment_measurements_fcn_val_full, sampled_values_and_filters_val_targets \
              = segment_gt(images, pairs, filters, use_fcn=use_fcn, split=splitl_val, return_sampled_values_and_filters=True)
      
      _, segment_measurements_fcn_val_few, _, _ = segment_measurements_fcn_val_few # 5 cmls
      _, segment_measurements_fcn_val_half, _, _ = segment_measurements_fcn_val_half # 10 cmls 
      _, segment_measurements_fcn_val_full, _, _ = segment_measurements_fcn_val_full # 20 cmls 
      sampled_values_and_filters_val_targets = sampled_values_and_filters_val_targets[2:]  #20 cmls
        
      # val step 1 : eval_opportunity_cost_spat
      step = 'eval_opportunity_cost_cml_spat'

      inputs = torch.cat([noisy_images,
                          0*segment_measurements_fcn_val_half
                          ], dim=1)
      outputs = model(inputs)

      regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, integ_loss = \
              criterion(model.p, outputs, point_measurements_fcn, sampled_values_and_filters=sampled_values_and_filters_val_targets)
                        
      running_regression_loss[step] += regression_loss
      running_regression_loss_1h[step] += regression_loss_1h
      running_segmentation_loss[step] += segmentation_loss
      running_confusion_matrix[step] += batch_cm
      running_integ_loss[step] += integ_loss
        
      del inputs, outputs, loss, regression_loss, regression_loss_1h, segmentation_loss
      torch.cuda.empty_cache()

      # val step 2 : eval_added_value_few_spat
      # step = 'eval_added_value_few_cml_spat'
      # inputs = torch.cat([noisy_images,
      #                     segment_measurements_fcn_val_few,
      #                     point_measurements_fcn_val_full
      #                     ], dim=1)
      # outputs = model(inputs)
      # regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, integ_loss = \
      #         criterion(model.p, outputs, point_measurements_fcn_val_targets, sampled_values_and_filters=sampled_values_and_filters_val_targets)
              
      # running_regression_loss[step] += regression_loss
      # running_regression_loss_1h[step] += regression_loss_1h
      # running_segmentation_loss[step] += segmentation_loss
      # running_confusion_matrix[step] += batch_cm
      # running_integ_loss[step] += integ_loss
        
      # del inputs, outputs, loss, regression_loss, regression_loss_1h, segmentation_loss, segment_measurements_fcn_val_few
      # torch.cuda.empty_cache()

      # val step 3 : eval_added_value_half_spat
      step = 'eval_added_value_half_cml_spat'
      inputs = torch.cat([noisy_images,
                          segment_measurements_fcn_val_half
                          ], dim=1)
      outputs = model(inputs)
      regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, integ_loss = \
      criterion(model.p, outputs, point_measurements_fcn, sampled_values_and_filters=sampled_values_and_filters_val_targets)   
      
      running_regression_loss[step] += regression_loss
      running_regression_loss_1h[step] += regression_loss_1h
      running_segmentation_loss[step] += segmentation_loss
      running_confusion_matrix[step] += batch_cm
      running_integ_loss[step] += integ_loss

      del inputs, outputs, loss, regression_loss, regression_loss_1h, segmentation_loss, segment_measurements_fcn_val_half
      torch.cuda.empty_cache()

      # val step 4 : eval_added_value_full_spat
      step = 'eval_added_value_full_cml_spat'
      inputs = torch.cat([noisy_images,
                          segment_measurements_fcn_val_full
                          ], dim=1)

      outputs = model(inputs)
      regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, integ_loss = \
              criterion(model.p, outputs, point_measurements_fcn, sampled_values_and_filters=sampled_values_and_filters_val_targets)  
              
      running_regression_loss[step] += regression_loss
      running_regression_loss_1h[step] += regression_loss_1h
      running_segmentation_loss[step] += segmentation_loss
      running_confusion_matrix[step] += batch_cm
      running_integ_loss[step] += integ_loss

      del inputs, loss, regression_loss, regression_loss_1h, segmentation_loss, segment_measurements_fcn_val_full
      torch.cuda.empty_cache()

      # last val step, on the 10 first pluvios : eval_added_value_full_id
      # step = 'eval_added_value_full_id'
      # regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, integ_loss = criterion(model.p, outputs, None,
      #                                                                                                segments=segment_measurements_fcn_eval_full) 
      # running_regression_loss[step] += regression_loss
      # running_regression_loss_1h[step] += regression_loss_1h
      # running_segmentation_loss[step] += segmentation_loss
      # running_confusion_matrix[step] += batch_cm
      # running_integ_loss[step] += integ_loss

      # del outputs, loss, regression_loss, regression_loss_1h, segmentation_loss, point_measurements_fcn_val_full, segment_measurements_fcn_val_full
      torch.cuda.empty_cache()

    # train step
    model.train()
    step = 'train'
    nl_train_inputs = torch.randint(0,19,(1,))
    nl_train_targets = npairs - nl_train_inputs

    # split train cmls
    splitl_train = [(0, nl_train_inputs), (nl_train_inputs, 20)]
    segment_measurements_fcn_train_inputs, sampled_values_and_filters_train_targets \
          = segment_gt(images, pairs, filters, use_fcn=use_fcn, split=splitl_train, return_sampled_values_and_filters=True)
  
    _, segment_measurements_fcn_train_inputs, _, _ = segment_measurements_fcn_train_inputs
    sampled_values_and_filters_train_targets = sampled_values_and_filters_train_targets[2:]


      
    inputs = torch.cat([noisy_images,
                        segment_measurements_fcn_train_inputs
                        ], dim=1)


    optimizer.zero_grad()  # Zero the gradients
    outputs = model(inputs)
    regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, integ_loss = criterion(model.p, outputs, 
                                                                                                   point_measurements_fcn,
                                                                                                   sampled_values_and_filters=sampled_values_and_filters_train_targets)

    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

    running_regression_loss[step] += regression_loss
    running_regression_loss_1h[step] += regression_loss_1h
    running_segmentation_loss[step] += segmentation_loss
    running_confusion_matrix[step] += batch_cm
    running_integ_loss[step] += integ_loss

    del inputs, outputs, loss, regression_loss, regression_loss_1h, segmentation_loss, noisy_images, images, pairs, filters, point_measurements_fcn, \
        segment_measurements_fcn_train_inputs, sampled_values_and_filters_train_targets
    torch.cuda.empty_cache()

  if epoch > 0:
    for step in steps:
      regression_loss = running_regression_loss[step] / len(loader)
      regression_loss_1h = running_regression_loss_1h[step] / len(loader)
      segmentation_loss = running_segmentation_loss[step] / len(loader)
      integ_loss = running_integ_loss[step] / len(loader)
      losses[step].append((epoch, regression_loss, regression_loss_1h, segmentation_loss, running_confusion_matrix[step], integ_loss))

      print(f'{step}, Regression Loss: {regression_loss:.4f}, Regression Loss 1h: {regression_loss_1h:.4f}, Segmentation Loss:{segmentation_loss:.4f}' )
      print("Train Confusion Matrix:")
      print(running_confusion_matrix[step])
      accuracy, csi, sensitivity, specificity, false_alarm_ratio = compute_metrics(running_confusion_matrix[step])
      print(f'Accuracy: {accuracy:.4f}, CSI: {csi:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, False Alarm Ratio: {false_alarm_ratio:.4f}')
      print(f'Integ Loss: {integ_loss:.4f}')
      print('\n')
  print('epoch duration :', time.time() - t)

  if (epoch % save_every == 0 or \
    epoch == last_epoch):
    print("saving step")
    checkpoint = { 
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        # 'scheduler': scheduler.state_dict(),
        'train_losses': losses,
        }
    torch.save(checkpoint, path)  
    

epoch n° 973 

eval_opportunity_cost_cml_spat, Regression Loss: 0.4117, Regression Loss 1h: 79.4030, Segmentation Loss:0.1582
Train Confusion Matrix:
[[3143618   52929]
 [ 128904  514549]]
Accuracy: 0.9526, CSI: 0.7389, Sensitivity: 0.7997, Specificity: 0.9834, False Alarm Ratio: 0.0933
Integ Loss: 0.0294


eval_added_value_half_cml_spat, Regression Loss: 0.4086, Regression Loss 1h: 78.9001, Segmentation Loss:0.1016
Train Confusion Matrix:
[[3140360   56187]
 [  92410  551043]]
Accuracy: 0.9613, CSI: 0.7876, Sensitivity: 0.8564, Specificity: 0.9824, False Alarm Ratio: 0.0925
Integ Loss: 0.0238


eval_added_value_full_cml_spat, Regression Loss: 0.4075, Regression Loss 1h: 78.7335, Segmentation Loss:0.0924
Train Confusion Matrix:
[[3144516   52031]
 [  85500  557953]]
Accuracy: 0.9642, CSI: 0.8023, Sensitivity: 0.8671, Specificity: 0.9837, False Alarm Ratio: 0.0853
Integ Loss: 0.0231


train, Regression Loss: 0.4080, Regression Loss 1h: 78.7125, Segmentation Loss:0.1021
Train Confusion M